# Transfer learning tests
Eamon Whalen

In [1]:
import sys
import os
import glob
import numpy as np
import pandas as pd
import altair as alt

sys.path.append('./models')
from feastnetSurrogateModel import FeaStNet
from pointRegressorSurrogateModel import PointRegressor

sys.path.append('./readers')
from loadGhGraphs import loadGhGraphs

sys.path.append('./visualization')
from altTrussViz import plotTruss, interactiveErrorPlot

sys.path.append('./util')
from gcnSurrogateUtil import *

## 1. Load test data

In [2]:
dataDir = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/"
testFile = os.path.join(dataDir, 'design_9_N_1000.csv')
allGraphsUnfiltered = loadGhGraphs(testFile, NUM_DV=5)

maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in allGraphsUnfiltered]
source = pd.DataFrame(maxes, columns=['maxes'])
source.describe()

,maxes
count,1000.000000
mean,0.141483
std,1.574372
min,0.006253
25%,0.021285
50%,0.035028
75%,0.072772
max,48.701946


In [3]:
testData = filterbyDisp(allGraphsUnfiltered, 0.9)
maxes = [max(np.abs(graph.y.numpy().flatten())) for graph in testData]
source = pd.DataFrame(maxes, columns=['maxes'])
maxDispCutoff = source.max().item()
source.describe()

,maxes
count,900.000000
mean,0.042389
std,0.030990
min,0.006253
25%,0.020052
50%,0.031303
75%,0.055108
max,0.140049


## 2. Load pre-train data

In [4]:
pretrainFiles = glob.glob(os.path.join(dataDir, '*1000.csv'))
pretrainFiles.remove(testFile)

allPretrainGraphs = []
for pretrainFile in pretrainFiles:
    designName = pretrainFile.split('/')[-1].split('_N')[0]
    print(f'loading {designName}')
    graphsUnfiltered = loadGhGraphs(pretrainFile, NUM_DV=5)
    graphs = filterbyDisp(graphsUnfiltered, 0.9)
    allPretrainGraphs.extend(graphs)

print(f'loaded {len(allPretrainGraphs)} pretraining graphs')
pretrainData, pretrainValData, _ = partitionGraphList(allPretrainGraphs, testSize=0.0, valSize=0.15)

loading design_7
loading design_6
loading design_8
loading design_5
loaded 3600 pretraining graphs


## 3. Initial training

In [5]:
saveDir = './results/transferLrn_des9_01/'
epochs=100
ptrGcn = FeaStNet()
history = ptrGcn.trainModel(pretrainData, pretrainValData, 
                            epochs=epochs,
                            saveDir=saveDir+f'preTrain/gcn/')

ptrGcnCheckptFile = ptrGcn.checkptFile
plotHistory(history)

epoch: 0   trainLoss: 7.2725e-01   valLoss:1.4253e+00  time: 7.37e+00
epoch: 1   trainLoss: 3.5379e-01   valLoss:6.7384e+00  time: 7.13e+00
epoch: 2   trainLoss: 2.4041e-01   valLoss:6.1685e-01  time: 7.00e+00
epoch: 3   trainLoss: 1.8522e-01   valLoss:4.6739e-01  time: 7.06e+00
epoch: 4   trainLoss: 1.4396e-01   valLoss:3.6726e-01  time: 7.04e+00
epoch: 5   trainLoss: 1.2312e-01   valLoss:3.2846e-01  time: 7.17e+00
epoch: 6   trainLoss: 1.1689e-01   valLoss:4.5505e-01  time: 6.97e+00
epoch: 7   trainLoss: 1.0619e-01   valLoss:2.7603e-01  time: 6.87e+00
epoch: 8   trainLoss: 1.0080e-01   valLoss:2.8771e-01  time: 6.91e+00
epoch: 9   trainLoss: 9.5196e-02   valLoss:4.2475e-01  time: 6.94e+00
epoch: 10   trainLoss: 9.9184e-02   valLoss:2.6734e-01  time: 6.96e+00
epoch: 11   trainLoss: 1.0495e-01   valLoss:1.6180e-01  time: 6.94e+00
epoch: 12   trainLoss: 1.1795e-01   valLoss:2.8249e-01  time: 7.04e+00
epoch: 13   trainLoss: 9.9028e-02   valLoss:1.3849e-01  time: 6.95e+00
epoch: 14   trai

alt.Chart(...)

In [6]:
trainRes = ptrGcn.testModel(pretrainData)
testRes = ptrGcn.testModel(testData) # unseen topology
pd.DataFrame([trainRes, testRes], index=['train', 'test'])

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2
train,0.000013,0.002188,0.090497,0.958374,NaN,NaN,NaN
test,0.000062,0.005307,0.172388,0.741564,0.933792,0.760291,-0.184816


## 4. Transfer learning study

In [7]:
dataDir = "/home/ewhalen/projects/data/trusses/2D_Truss_v1.3/"
trainDataFiles = glob.glob(os.path.join(dataDir, 'design_9*.csv'))
trainDataFiles.remove(testFile)

allResults = []
for trainDataFile in trainDataFiles:
    trainDataUnfiltered = loadGhGraphs(trainDataFile, NUM_DV=5)
    trainData = filterbyDispValue(trainDataUnfiltered, maxDispCutoff)
    trainSize = len(trainData)
    print(f'loaded train set of size {trainSize}')
    
    
    ### fresh neural network ###
    gcn = FeaStNet()
    history = gcn.trainModel(trainData, trainData, 
                         epochs=epochs, 
                         saveDir=saveDir+f'{trainSize}/gcn/')
    
    trainRes = gcn.testModel(trainData)
    trainRes['Model'] = 'Fresh'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = gcn.testModel(testData)
    testRes['Model'] = 'Fresh'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)

    
    ### transfer learning ###
    ptrGcn = FeaStNet()
    history = gcn.trainModel(trainData, trainData, 
                             restartFile=ptrGcnCheckptFile,
                             epochs=epochs, 
                             saveDir=saveDir+f'{trainSize}/ptrGcn/')
    
    trainRes = gcn.testModel(trainData)
    trainRes['Model'] = 'Transfer learning'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = gcn.testModel(testData)
    testRes['Model'] = 'Transfer learning'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)
    
    
    ### random forest ###
    rf = PointRegressor('Random Forest')
    rf.trainModel(trainData, trainData, 
                     saveDir=saveDir+f'{trainSize}/rf/')

    trainRes = rf.testModel(trainData)
    trainRes['Model'] = 'Random Forest'
    trainRes['Set'] = 'Train'
    trainRes['Train Size'] = trainSize
    allResults.append(trainRes)
    
    testRes = rf.testModel(testData)
    testRes['Model'] = 'Random Forest'
    testRes['Set'] = 'Test'
    testRes['Train Size'] = trainSize
    allResults.append(testRes)
    pd.DataFrame(allResults).to_csv(saveDir+'testResults.csv', index=False)
    
    
pd.DataFrame(allResults)

loaded train set of size 48
epoch: 0   trainLoss: 9.4186e-01   valLoss:9.5560e-01  time: 4.32e-01
epoch: 1   trainLoss: 7.7727e-01   valLoss:9.4003e-01  time: 4.42e-01
epoch: 2   trainLoss: 6.6229e-01   valLoss:9.1578e-01  time: 4.50e-01
epoch: 3   trainLoss: 5.6774e-01   valLoss:8.8743e-01  time: 4.48e-01
epoch: 4   trainLoss: 4.6349e-01   valLoss:8.5494e-01  time: 4.59e-01
epoch: 5   trainLoss: 3.9376e-01   valLoss:8.2469e-01  time: 4.30e-01
epoch: 6   trainLoss: 3.2349e-01   valLoss:8.1825e-01  time: 4.33e-01
epoch: 7   trainLoss: 2.6142e-01   valLoss:8.3949e-01  time: 4.34e-01
epoch: 8   trainLoss: 2.1795e-01   valLoss:8.8210e-01  time: 4.33e-01
epoch: 9   trainLoss: 1.9015e-01   valLoss:9.3346e-01  time: 4.39e-01
epoch: 10   trainLoss: 1.7563e-01   valLoss:9.5297e-01  time: 4.31e-01
epoch: 11   trainLoss: 1.5534e-01   valLoss:9.5408e-01  time: 4.28e-01
epoch: 12   trainLoss: 1.4607e-01   valLoss:9.2513e-01  time: 4.29e-01
epoch: 13   trainLoss: 1.3889e-01   valLoss:9.3186e-01  tim

epoch: 15   trainLoss: 9.7771e-02   valLoss:3.1456e-01  time: 4.24e-01
epoch: 16   trainLoss: 8.9025e-02   valLoss:2.6532e-01  time: 4.23e-01
epoch: 17   trainLoss: 7.8687e-02   valLoss:2.2505e-01  time: 4.21e-01
epoch: 18   trainLoss: 7.8339e-02   valLoss:1.7222e-01  time: 4.24e-01
epoch: 19   trainLoss: 6.8303e-02   valLoss:2.1605e-01  time: 4.24e-01
epoch: 20   trainLoss: 6.7263e-02   valLoss:1.8227e-01  time: 4.25e-01
epoch: 21   trainLoss: 5.8265e-02   valLoss:9.0417e-02  time: 4.26e-01
epoch: 22   trainLoss: 5.0787e-02   valLoss:6.4769e-02  time: 4.27e-01
epoch: 23   trainLoss: 5.2088e-02   valLoss:9.8280e-02  time: 4.20e-01
epoch: 24   trainLoss: 4.7897e-02   valLoss:1.2745e-01  time: 4.25e-01
epoch: 25   trainLoss: 4.3389e-02   valLoss:1.1500e-01  time: 4.16e-01
epoch: 26   trainLoss: 4.2176e-02   valLoss:1.0468e-01  time: 4.20e-01
epoch: 27   trainLoss: 3.7702e-02   valLoss:1.1417e-01  time: 4.26e-01
epoch: 28   trainLoss: 3.7316e-02   valLoss:1.1364e-01  time: 4.24e-01
epoch:

epoch: 31   trainLoss: 1.4972e-02   valLoss:2.6096e-01  time: 1.71e-01
epoch: 32   trainLoss: 1.5520e-02   valLoss:2.6135e-01  time: 1.73e-01
epoch: 33   trainLoss: 1.4271e-02   valLoss:2.8526e-01  time: 1.72e-01
epoch: 34   trainLoss: 1.1651e-02   valLoss:2.7952e-01  time: 1.72e-01
epoch: 35   trainLoss: 1.0301e-02   valLoss:2.8700e-01  time: 1.76e-01
epoch: 36   trainLoss: 8.4198e-03   valLoss:2.7419e-01  time: 1.76e-01
epoch: 37   trainLoss: 6.9203e-03   valLoss:2.6867e-01  time: 1.78e-01
epoch: 38   trainLoss: 5.7935e-03   valLoss:2.5334e-01  time: 1.72e-01
epoch: 39   trainLoss: 6.1147e-03   valLoss:2.2850e-01  time: 1.73e-01
epoch: 40   trainLoss: 5.3042e-03   valLoss:2.3038e-01  time: 1.74e-01
epoch: 41   trainLoss: 5.3936e-03   valLoss:2.2788e-01  time: 1.74e-01
epoch: 42   trainLoss: 4.4071e-03   valLoss:2.3003e-01  time: 1.73e-01
epoch: 43   trainLoss: 3.7351e-03   valLoss:2.1957e-01  time: 1.75e-01
epoch: 44   trainLoss: 3.3481e-03   valLoss:2.0760e-01  time: 1.72e-01
epoch:

epoch: 47   trainLoss: 3.6137e-03   valLoss:2.5914e-02  time: 1.79e-01
epoch: 48   trainLoss: 3.4810e-03   valLoss:2.0141e-02  time: 1.74e-01
epoch: 49   trainLoss: 3.3515e-03   valLoss:1.6957e-02  time: 1.76e-01
epoch: 50   trainLoss: 3.2261e-03   valLoss:1.9338e-02  time: 1.76e-01
epoch: 51   trainLoss: 3.2332e-03   valLoss:2.0490e-02  time: 1.75e-01
epoch: 52   trainLoss: 3.2244e-03   valLoss:2.7332e-02  time: 1.75e-01
epoch: 53   trainLoss: 3.2978e-03   valLoss:2.4031e-02  time: 1.78e-01
epoch: 54   trainLoss: 3.6553e-03   valLoss:3.4863e-02  time: 1.89e-01
epoch: 55   trainLoss: 4.5812e-03   valLoss:2.8094e-02  time: 1.77e-01
epoch: 56   trainLoss: 6.6294e-03   valLoss:4.0125e-02  time: 1.77e-01
epoch: 57   trainLoss: 9.1923e-03   valLoss:2.7894e-02  time: 1.78e-01
epoch: 58   trainLoss: 9.2887e-03   valLoss:2.5544e-02  time: 1.74e-01
epoch: 59   trainLoss: 4.1530e-03   valLoss:1.5860e-02  time: 1.75e-01
epoch: 60   trainLoss: 3.0343e-03   valLoss:2.2992e-02  time: 1.76e-01
epoch:

epoch: 62   trainLoss: 4.1508e-02   valLoss:2.2562e-01  time: 3.86e+00
epoch: 63   trainLoss: 3.4572e-02   valLoss:3.5142e-01  time: 3.80e+00
epoch: 64   trainLoss: 3.6134e-02   valLoss:2.4152e-01  time: 3.81e+00
epoch: 65   trainLoss: 3.2443e-02   valLoss:1.6048e-01  time: 3.86e+00
epoch: 66   trainLoss: 3.4776e-02   valLoss:1.9285e-01  time: 3.89e+00
epoch: 67   trainLoss: 2.5899e-02   valLoss:1.8871e-01  time: 3.81e+00
epoch: 68   trainLoss: 2.7614e-02   valLoss:2.0747e-01  time: 3.84e+00
epoch: 69   trainLoss: 2.2523e-02   valLoss:2.0548e-01  time: 3.87e+00
epoch: 70   trainLoss: 2.1115e-02   valLoss:2.2073e-01  time: 3.84e+00
epoch: 71   trainLoss: 4.1177e-02   valLoss:1.5243e-01  time: 3.89e+00
epoch: 72   trainLoss: 3.0718e-02   valLoss:1.4279e-01  time: 3.85e+00
epoch: 73   trainLoss: 2.8420e-02   valLoss:1.4602e-01  time: 3.84e+00
epoch: 74   trainLoss: 3.3931e-02   valLoss:1.5412e-01  time: 3.86e+00
epoch: 75   trainLoss: 2.7623e-02   valLoss:1.7420e-01  time: 3.86e+00
epoch:

...

epoch: 54   trainLoss: 1.5001e-02   valLoss:4.2860e-02  time: 8.11e-01
epoch: 55   trainLoss: 9.2667e-03   valLoss:4.4487e-02  time: 8.02e-01
epoch: 56   trainLoss: 1.1861e-02   valLoss:3.5914e-02  time: 7.84e-01
epoch: 57   trainLoss: 9.3267e-03   valLoss:3.1037e-02  time: 8.14e-01
epoch: 58   trainLoss: 9.9017e-03   valLoss:2.8121e-02  time: 7.88e-01
epoch: 59   trainLoss: 8.5214e-03   valLoss:2.6441e-02  time: 7.91e-01
epoch: 60   trainLoss: 8.3902e-03   valLoss:3.1398e-02  time: 8.01e-01
epoch: 61   trainLoss: 8.0871e-03   valLoss:2.5511e-02  time: 8.17e-01
epoch: 62   trainLoss: 6.9744e-03   valLoss:1.8661e-02  time: 7.95e-01
epoch: 63   trainLoss: 7.7141e-03   valLoss:1.9832e-02  time: 7.91e-01
epoch: 64   trainLoss: 6.1881e-03   valLoss:1.7499e-02  time: 7.88e-01
epoch: 65   trainLoss: 6.6149e-03   valLoss:1.5491e-02  time: 8.24e-01
epoch: 66   trainLoss: 6.1326e-03   valLoss:1.6207e-02  time: 8.04e-01
epoch: 67   trainLoss: 5.6334e-03   valLoss:1.8924e-02  time: 7.98e-01
epoch:

epoch: 69   trainLoss: 2.8861e-02   valLoss:4.3609e-02  time: 7.88e+00
epoch: 70   trainLoss: 4.5674e-02   valLoss:5.5100e-02  time: 7.85e+00
epoch: 71   trainLoss: 3.5261e-02   valLoss:4.7679e-02  time: 7.78e+00
epoch: 72   trainLoss: 3.4419e-02   valLoss:4.9637e-02  time: 7.82e+00
epoch: 73   trainLoss: 3.9823e-02   valLoss:8.7582e-02  time: 7.82e+00
epoch: 74   trainLoss: 4.1319e-02   valLoss:5.1229e-02  time: 7.73e+00
epoch: 75   trainLoss: 4.0186e-02   valLoss:7.7679e-02  time: 7.84e+00
epoch: 76   trainLoss: 3.4270e-02   valLoss:5.4962e-02  time: 7.84e+00
epoch: 77   trainLoss: 4.8045e-02   valLoss:4.7933e-02  time: 7.81e+00
epoch: 78   trainLoss: 4.6611e-02   valLoss:4.2696e-02  time: 7.82e+00
epoch: 79   trainLoss: 4.0320e-02   valLoss:5.8746e-02  time: 7.85e+00
epoch: 80   trainLoss: 3.4100e-02   valLoss:5.7013e-02  time: 7.77e+00
epoch: 81   trainLoss: 3.4612e-02   valLoss:7.3466e-02  time: 7.85e+00
epoch: 82   trainLoss: 3.6770e-02   valLoss:8.4578e-02  time: 7.89e+00
epoch:

epoch: 84   trainLoss: 4.0260e-02   valLoss:1.2618e-01  time: 7.69e+00
epoch: 85   trainLoss: 4.3327e-02   valLoss:9.6933e-02  time: 7.81e+00
epoch: 86   trainLoss: 4.7483e-02   valLoss:1.9842e-01  time: 7.77e+00
epoch: 87   trainLoss: 5.0297e-02   valLoss:6.3483e-02  time: 7.83e+00
epoch: 88   trainLoss: 5.1200e-02   valLoss:6.6966e-02  time: 7.81e+00
epoch: 89   trainLoss: 3.6984e-02   valLoss:8.2331e-02  time: 7.77e+00
epoch: 90   trainLoss: 4.6055e-02   valLoss:2.6859e-01  time: 7.79e+00
epoch: 91   trainLoss: 5.2479e-02   valLoss:2.3410e-01  time: 7.75e+00
epoch: 92   trainLoss: 4.3168e-02   valLoss:1.1824e-01  time: 7.82e+00
epoch: 93   trainLoss: 3.9220e-02   valLoss:7.1859e-02  time: 7.82e+00
epoch: 94   trainLoss: 3.8190e-02   valLoss:1.4837e-01  time: 7.82e+00
epoch: 95   trainLoss: 3.8246e-02   valLoss:6.0340e-02  time: 7.74e+00
epoch: 96   trainLoss: 4.2598e-02   valLoss:5.6555e-02  time: 7.75e+00
epoch: 97   trainLoss: 3.2448e-02   valLoss:8.1798e-02  time: 7.77e+00
epoch:

loading restart file
epoch: 0   trainLoss: 3.0883e-01   valLoss:1.3243e+00  time: 8.52e-02
epoch: 1   trainLoss: 3.6212e-01   valLoss:7.8721e+00  time: 8.64e-02
epoch: 2   trainLoss: 1.7104e-01   valLoss:6.0023e-01  time: 8.88e-02
epoch: 3   trainLoss: 9.3814e-02   valLoss:8.1151e-01  time: 8.99e-02
epoch: 4   trainLoss: 6.5106e-02   valLoss:6.3800e-01  time: 8.92e-02
epoch: 5   trainLoss: 4.9501e-02   valLoss:3.7689e-01  time: 9.09e-02
epoch: 6   trainLoss: 4.4609e-02   valLoss:2.8836e-01  time: 9.14e-02
epoch: 7   trainLoss: 3.9954e-02   valLoss:2.5355e-01  time: 9.46e-02
epoch: 8   trainLoss: 3.7229e-02   valLoss:1.7444e-01  time: 8.78e-02
epoch: 9   trainLoss: 2.8229e-02   valLoss:1.3386e-01  time: 8.58e-02
epoch: 10   trainLoss: 2.6884e-02   valLoss:1.2538e-01  time: 8.82e-02
epoch: 11   trainLoss: 2.0044e-02   valLoss:1.2681e-01  time: 8.81e-02
epoch: 12   trainLoss: 1.5721e-02   valLoss:1.1222e-01  time: 8.63e-02
epoch: 13   trainLoss: 1.4419e-02   valLoss:8.1013e-02  time: 8.63

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2,Model,Set,Train Size
0,0.000045,0.004630,0.163317,0.882675,0.968082,0.493723,-11.255813,Fresh,Train,48
1,0.000132,0.007395,0.236134,0.478429,0.766847,0.359519,-6.779388,Fresh,Test,48
2,0.000006,0.001685,0.061272,0.976042,0.994296,0.961505,0.407564,Transfer learning,Train,48
3,0.000067,0.004643,0.148968,0.690041,0.883682,0.762376,-0.028446,Transfer learning,Test,48
4,0.000022,0.002845,0.090335,0.916350,0.977951,0.949572,0.850276,Random Forest,Train,48
5,0.000112,0.006616,0.206737,0.536838,0.777957,0.673963,0.530883,Random Forest,Test,48
6,0.000030,0.003575,0.120591,0.930699,0.993558,0.841346,-0.819952,Fresh,Train,18
7,0.000218,0.009086,0.269193,0.188515,0.660763,0.109233,-9.802806,Fresh,Test,18
8,0.000005,0.001649,0.067048,0.980933,0.995591,0.967842,0.736369,Transfer learning,Train,18
9,0.000111,0.005995,0.180815,0.466764,0.792074,0.655240,0.169769,Transfer learning,Test,18


In [8]:
df = pd.DataFrame(allResults)
df

,mse,mae,mre,peakR2,maxAggR2,meanAggR2,minAggR2,Model,Set,Train Size
0,0.000045,0.004630,0.163317,0.882675,0.968082,0.493723,-11.255813,Fresh,Train,48
1,0.000132,0.007395,0.236134,0.478429,0.766847,0.359519,-6.779388,Fresh,Test,48
2,0.000006,0.001685,0.061272,0.976042,0.994296,0.961505,0.407564,Transfer learning,Train,48
3,0.000067,0.004643,0.148968,0.690041,0.883682,0.762376,-0.028446,Transfer learning,Test,48
4,0.000022,0.002845,0.090335,0.916350,0.977951,0.949572,0.850276,Random Forest,Train,48
5,0.000112,0.006616,0.206737,0.536838,0.777957,0.673963,0.530883,Random Forest,Test,48
6,0.000030,0.003575,0.120591,0.930699,0.993558,0.841346,-0.819952,Fresh,Train,18
7,0.000218,0.009086,0.269193,0.188515,0.660763,0.109233,-9.802806,Fresh,Test,18
8,0.000005,0.001649,0.067048,0.980933,0.995591,0.967842,0.736369,Transfer learning,Train,18
9,0.000111,0.005995,0.180815,0.466764,0.792074,0.655240,0.169769,Transfer learning,Test,18


In [10]:
alt.Chart(df[df.Set=='Test']).mark_circle().encode(
    x=alt.X('Train Size:Q', scale=alt.Scale(type='log')),
    y=alt.Y('mse:Q', title='MSE'),
    color='Model',
    tooltip=['Model', 'mse']
)

alt.Chart(...)